# Milestone 1
## Team 31


------

### Introduction

Automatic differentiation is a widely used tool for optimization, encompassing a set of efficient techniques used to computationally evaluate a function's derivative or gradient. At it's core, automatic differentation exploits the idea that every function can be broken down into a sequence of elemenatry arithmetic operations and functions. Applying the chain rule to these operations in succession allows us to evaluate the derivatives we seek to high precision, without rampantly growing the number of arithmetic operations the computer needs to perform. As functions grow in size, complexity, and dimension, the need for an efficient algorithm to accurately derive the gradient becomes clear. The goal of our project is to develop a software library for automatic differentiation with both a forward mode and a reverse mode implementation to provide such efficient and accurate derivative calculations.

The applications of automatic differentiation are vast and ever-expanding. Ranging from finding the solution to ordinary differential equations (ODEs), to optimizing objective functions, to finding the solution of linear systems as an alternative to iterative methods, automatic differentiation is a highly used technique. Popular machine learning libraries such as Pytorch even make use of such automatic differentiation techniques. Prominent example applications span essentially any gradient-based optimization algorithm, including Newton’s method, training neural networks, sampling high-dimensional Bayesian model posteriors, and  Hamiltonian Monte Carlo samplers.$^{(1)}$ Speeding up computation and eliminating round-off error is crucial in applying modern gradient-based optimization algorithms across science and engineering. Automatic differentiation, as opposed to alternatives such as symbolic and numerical differentiation, holds many advantages in terms of both its efficiency and accuracy. As models – especially in neural network based machine learning – become more complex and high-dimensional data becomes more readily available, the ability efficiently to compute derivatives is only becoming more important.

$^{(1)}$ *A Review of Automatic Differentiation and its Efficient Implementation* (Margossian)

------

### Background

#### Forward Mode
Automatic differentiation is a set of techniques to computationally determine a function's gradient, centered around an iterative evaluation of the chain rule.  This piecewise evaluation of the chain rule to find a derivative stems from the idea of breaking down complex functions into a sequence of elementary operations (addition, multiplication, etc.) and elementary functions (sin, exp, etc.), from the inside of the function outwards.  This sequence of elementary operations and functions that string together to create the function definition give rise to the forward primal trace of the function.  Using our function inputs and evaluating through this sequence, we can see that at each stage in the sequence we will get an intermediate result $v_j$ that is dependent on our inputs.

The breakdown of a complex function into this sequence of elementary operations and functions (from the inside out) is most easily visualized in what we call the computational forward graph.  Our inputs $x_k$ to the original complex function represents nodes $v_{k-m}$ for $k$ = 1,2,...,m.  As we pass through the sequence, each intermediate results $v_j$ is alloted a node while the directed edges connecting nodes represent the elementary operation/function that leads to in that intermediate result (a new node on the graph).

To compute the given derivative of the complex function we need to define a differential operator $D_p(.)$ representing the directional derivative in the direction of $p$ (note: $p$ is the seed vector that the user chooses if they desire a certain derivative of interest). Then, when we evaluate the forward primal trace of the function (finding all the intermediate values $v_j$), we also simultaneously evaluate (and carry forward) $D_p v_j$, the derivative of $v_j$ in direction of $p$.  Thus, for each step in the sequence we get both an intermediate value $v_j$ based on previous $v_j$'s  from the forward primal trace but also a value $D_p v_j$ based on the previous $D_p v_j$'s, (note: the sequence of formulas that define $D_p v_j$ at every step for a given value of p is called the forward tangent trace). Every time we wish to compute the derivative in a new direction we must perform a new pass through the forward tangent trace, plugging in a different $p$ direction vector to our $D_p v_j$'s at each step to find their values.

An important aside in the calculation of our gradients is the application of the chain rule touched on above. The chain rule allows the representation of a functions derivative in terms of the composition of multiple differentiable functions. Beginning with a simple example, we can consider a function in the form $f(g(t))$, and as a result of the chain rule write its derviative as $\frac{df}{dt} = \frac{df}{dg} \cdot \frac{dg}{dt}$. We can expand this concept to higher dimensions to further generalize our chain rule. With a multivariate function, it is important to consider the gradient, which represents the summation of the partial derivatives of f to its inputs. We can thus write our generalized chain rule as follows: $\nabla_x f = \sum_{i=1}^n \frac{\partial f}{\partial y_i} \nabla y_i (x)$.

One structure that is critical in the implementation of forward mode AD is dual numbers.  A dual number $z$ takes the form $z = a + b\epsilon$ where $a$ is the real part, $b$ is the dual part, and $\epsilon$ is a nilpotent number such that $\epsilon^2 = 0$ (note: $\epsilon$ does not equal 0). Because of the following three useful properties:
1. Adding dual numbers together resembles the linearity of addition and results in adding the functionsin the real part and adding the derivatives in the dual part
2. Multiplication results in multiplication of the functions in the real part and the product rule for the derivatives in the dual part
3. Chain rule still holds for dual numbers

dual numbers can be used to efficiently encode the primal and tangent trace of intermediate results $v_i$ and $v_j$ as we pass forward through the sequence (computational graph).  Letting $z_j = v_j + D_p v_j \epsilon$ for intermediate results, we are able to conveniently store the primal trace and the tangent trace in the real and dual part (respectively) of a dual number ($z_j$) allowing us to carry them forward through the sequence as a pair.


#### REVERSE MODE
While the forward mode implementation described above holds many advantages for the evaluation of a function and its gradient, it has shortcomings when dealing with functions of high dimensions. In particular, for functions $f: \mathbb{R}^m:\mathbb{R}$ for large $m$, forward mode requires $m$ passes to determine the gradient of $f$. This is a common scenario for neural networks, and is where the alternative technique of reverse mode excels.

Reverse mode, as opposed to forward mode, requires only two passes to determine the gradient of a function. The first pass, named the forward pass, computes the primal trace $(v_j)$ as well as the partial derivatives with respect to the parent nodes. However, unlike in forward mode, the chain rule must not be explicitly applied. Instead, we then perform the reverse pass, where the chain rule will be reconstructed by accumulating the values as we traverse backwards across our computational graph. The result is a calculation of the gradient of c, with a computational cost not dependent on $m$ as forward mode is. However, reverse mode does have limitations in the sense that we must store the computational graph of our function which may be memory intensive. For a forward mode implementation, we iterate over the graph sequentially and do not need to return to prior nodes, whereas for the reverse pass in our reverse mode implementation these previous nodes must be stored as they are later revisited. Hence, while reverse mode may require more memory allocation in terms of storing the computational graph of our function, it nonetheless offers a substantial advantage in terms of its computational cost for high order functions and is thus an extension of our automatic differentiation library.






-----

### How to Use AutoDiff

We expect users to interact with our package in roughly the same way they interact with other, popular packages that they may already be familiar with (numpy, scipy, etc.).  Our package will be published on Python Package Index (PyPi) and should be installed using pip with a command like *python -m pip install autodiff*. 

Our package will be organized in a hierarchical structure with our autodiff package at the highest level and multiple subpackages, each with different modules, beneath it (see software organization and implementation for further details regarding contents of subpackages and modules). 

By importing the autodiff package (eg. *import autodiff as ad*), users should have access to the main package as well as subpackages and modules (this will be further defined in our \_\_init\_\_.py files).  With *import autodiff as ad*, users should be able to access functions, classes, etc. from the main module with *ad.function* for example, or from subpackages with *ad.subpackage.function*.  Users could also import subpackages directly with *import autodiff.subpackage as sub* and then call *sub.function* for example.  They should be able to instantiate AD objects in the typical way as well.  They could *import autodiff as ad* and call *instance = ad.class()* or use *from AutomaticDifferentiation import class* and call *instance = class()* for example.

Specific to our implementation, after importing the autodiff package users can first define a dual number by calling the DualNumber class and specifying the real and dual components. An example would be as follows:

```python
z1 = ad.DualNumber(a_1,b_1)

z2 = ad.DualNumber(a_2,b_2)
```

In this case, a represents the point at which they wish to evaluate the function, while b represents the component of the seed vector.

After instantiating such an object, users must define the function they wish to derive. We plan to incorporate a high level function class in which users may input their desired function as well as a the number of inputs and outputs of the function. An example of such a function initialization would be as follows:

```python
def f(x,y): return x**2+y+1

f1 = ad.func(f, num_inputs=2, num_outputs=1)
```

Users can then directly evaluate the gradient at a specific point without the specification of any seed vectors by calling the grad method of their function ($f1.grad()$). Alternatively, users can also choose to evaluate their function at a particular seed vector and value, using $f1.eval()$ and passing in a dual number. 

This will return the evaluation of the function $f1$, as well as the derivative in the direction of the seed vector (and therefore will be dependent on the dual numbers passed in).

Another possible case is with our reverse mode implementation for our extension. In this case users can similarly define functions and dual numbers, but must specify rev_func.method() upon calling either the grad() or eval() methods. Additionally, there is the option of visualizing their graph. Users may use their function objects and call the .graph() method, passing in a point and a seed vector.

### Example

Basic Forward Mode:

```python
>>> import autodiff as ad
>>> def f(x,y): return 2*x+y+1
>>> f1 = ad.func(f, num_inputs=2, num_outputs=1)
>>> f1.eval(p = [1, 3], seed = [2,4])
DualNumber(6, 8) #real part is function evaluation at p and dual part is derivative in direction seed at p
>>> f1.grad(p = [1,1])
[[2],[1]] #the value of the function gradient at the point p
```

For Reverse Mode and Graph Illustration:

```python
>>> import autodiff.reverse as rev
>>> from autodiff.reverse import rev_dualnumber as RevDualNumber
>>> x1, x2 = (1,1)
>>> input1 = RevDualNumber(x1)
>>> input2 = RevDualNumber(x2)
>>> res = f(input1,input2)

>>> res.real
4 #the value of our function evaluate at the point (x1,x2)
>>> res.backwards()
[[2],[1]] #the value of the function gradient at the point (x1,x2)

>>> from autodiff.reverse import graph
>>> graph(res)

{computational graph displayed}

```

-------

### Software Organization


Our directory structure will look like the following, where autodiff is the name of our main package that clients will be importing, with subpackages specifice to forward and reverse mode automatic differentiation.
```
team31 
├── LICENSE    
├── pyproject.toml      
├── README.md
├── setup.cfg
├── docs
│    └── \*
├── src
│    └── autodiff  # main package
│            ├── __init__.py 
│            ├── func.py 
│            ├── operators.py 
│            ├── dualnumber.py  
│            └── reverse
│                   ├── __init__.py 
│                   ├── rev_dualnumber.py
│                   ├── graph.py 
└── test
      └── autodiff  
             ├── test_dualnumber.py  
             ├── test_func.py 
             ├── test_operators.py
             └── reverse
                   ├── test_rev_dualnumber.py
                   ├── test_graph.py

```

Firstly, for our basic implementation we plan on including a dualNumber module. Within this module we will define a DualNumber class, where we will initiate such numbers and overwrite all operators for basic arithmetic such as +, -,*, /. We will also have checks for the type of the instance when we attempt one of these operations, as we intend to provide support for int and float operations paired with dual numbers. Similarly, within our operators.py module, we will override operators: sin, cos, log, and pow such that users can generate functions using our implementations of these unary operators. Additionally, we currently intend on creating a high level function module (func.py) in which users can pass in their functions as well as the input and output dimenions (in the form of (f, num\_inputs, num\_outputs)). Within the func.py Func class we also explicitly define the grad() and eval() methods for forward mode. Similarly, within our reverse mode subpackage, we override these functions inherit the Func class and override its methods within rev_func.py, in accordance with the reverse mode implementation. Lastly, within the reverse subpackage, we also have a rev_dual.py module which modifies our definition of dual numbers and allows us to store the parents of particular nodes to maintain our computational graph structure.

We will store our test suite separate from our source code, but it will mirror the directory structure used in the source code, as seen in the above directory diagram. We intend on implementing a range of unit tests, integration tests, and system tests using pytest to ensure a high level of code coverage and proper implementation of all functionality of our project.

We will distribute our package via PyPI with PEP517/518. The inclusion of the pyproject.toml file within our project is a necessary requirement for publication on PyPI (PEP 518). This file will specify the minimum system requirements and contain some project metadata. Additionally, within our setup.cfg we will specify the build configuration for the setuptools backend that will be used when installing our package via pip. This will enable the easy building and distribution of our project. Moreover, it will allow us to create new releases if we make any updates to our project.

Additional considerations would include any dependencies we anticipate requiring. We anticipate using numpy as well as graphviz (for our graph visualization). Thus, within our setup.cfg file we will specify that our package requires both numpy and graphviz.



-------

### Implementation

#### DualNumber()
The cornerstone of our AD package is the base class DualNumber in the module dualnumber.py.  DualNumber takes two inputs, real – the real part of the dual number and dual – the dual part of the number (initlized to 1 if nothing is passed). These will be stored in attributes self.real and self.dual. When implementing forward mode AD we will use this base DualNumber class directly.  A useful first method would be the \_\_repr\_\_ method which will return a printable string that describes the dual number such as "DualNumber(real={self.real}, dual={self.dual})".  Because we will be calling elementary functions on our dual numbers to find paired values of the function and the derivative, in the methods of this class we will overload all the basic operators so that they reflect what it would be like to perform those operations on dual numbers.  For example, we will define dunder methods for \_\_add\_\_, \_\_sub\_\_, \_\_mul\_\_, etc.; to be thorough we will also define \_\_radd\_\_,  \_\_rsub\_\_, and \_\_rmul\_\_. Additionally, we will make careful consideration of the allowed instance types for such functions. We intend to allow users to perform operations with dual numbers and ints and floats. Thus, within overriding the above functions, we will check for the type of the other element being operated with. We will have separate cases when adding (or performing another operation) an int or float to a dual number, and properly account for such instances. Instances of this class are going to be initialized based on point and seed vector inputs and used in forward mode AD to compute values and deriavtives.
 

We also need to define methods for all elementary functions such as sin, cos, log, exp, \_\_pow\_\_ (a dunder method) which we can store within this DualNumber class as well (as opposed to a separate module for overriding such operators).  For example, the sin method will return DualNumber(np.sin(self.real), np.cos(self.dual) * self.dual), a new dual number that represents sin of the dual number input. We can apply our knowledge of the chain rule for overriding such operators, where the real component of the resulting dual number is simply the operator applied to the real component input, while the dual component is the chain rule applied to the real and dual components of the input. 
#### Func()
The other major class we have is the base class Func in the module func.py. This function will take 3 inputs: func, num_inputs, num_outputs. Func is the user defined function and will be stored as an attribute in self.func (since functions are first class objects within python).  num_inputs and num_outputs represent the respective input and output dimension of self.func and will be stored in attriuted self.num_inputs and self.num_outputs.  This base class will also have two methods grad() and eval(). 

The method grad() will take a point p as input where p is a sequence of scalars (eg. p = [1,3,4,2], note: p has dimension self.num_inputs and this must be checked).  This is the point where users want to evaluate the gradient of the function stored in the attribute self.func.  The grad() method will then create an identity matrix of dimension num_inputs by num_inputs where each column represents a seed vector. We will then create a for loop that iterates through the columns of this matrix, calling each column seed_vector. In each loop we will initialize num_inputs dual numbers from the dual number class where dual number i is: $z_i$ = DualNumber(p[i],seed_vector[i]).  We will then call self.func($z_1$...$z_{num\_inputs}$) which will return a dual number where its real part is the value of the function at the input point p and its dual part is the derivative of the function at point p in direction seed_vector.  We will then store this derivative value or vector if there are multiple outputs – as a column in a self.num_outputs by self.num_inputs matrix called gradient.  Once we have looped through all the seed_vectors (and thus all the inputs), we will output this gradient matrix which will hold the values of the derivative of each output of self.func with respect to each input of self.func at the point p.

The other method of func is eval(). eval() allows users to find the value of the function self.func and the value deriative in a specific user-defined direction at a given point.  eval() will take two inputs, a point p (the point where users want to evaluate the function and derivative) and a direction (seed_vector), the direction in which users want to evaluate the direction derivative (eg. seed_vector = [1,0,0,0]).  The eval method will again define num_inputs dual numbers $z_i$, where $z_i$ = DualNumber(p[i],seed_vector[i]).  Note, the elements of the inputted point vector and seed direction vector are scalar (ints or floats).  The eval method will then call and return self.func($z_1$...$z_{num\_inputs}$).  This means the eval method will return a dual number where the real part is the value of the function evaluated at the point p and the dual part is the derivative of the function in the direction seed_vector evaluated at the point p.

#### Reverse Mode Extension
For reverse mode AD we will have a subpackage within our general AD package.  Because for reverse mode AD we need the function evaluation in order to compute gradients in reverse, the primal trace of the graph so we can traverse it backwards, and the operations neccessary to compute the gradient, we will have to alter our base class functions that are compatible with forward mode AD to make them more powerful.  For this reason we define a new class RevDualNumber, in the reverse subpackage in the rev_dualnumber.py module, which inherits from our DualNumber class.  The class will be instantiated with the same inputs as DualNumber class but with parents and operation inputs as well.  These will become supplementary attributes which are needed to create a computational graph that will allow us to perform the reverse pass.  We keep an create an attribute self.op to keep track of the operation type performed on with that dual number, a self.parents attribute to keep track of the parents of that dual number, and a self.name attribute that defines the intermediate result $v_j$ (which becomes the node on the graph) represented by the dual number.  This forces us to redefine our some dunder and overloading methods within the class as well, such that our new methods will instantiate the new dual numbers representing intermediate nodes correctly.  

We also have to add a backwards function to all of our overloaded methods that represent the derivative of the elementary functions defined by each overloaded method.  By keeping track of the names of nodes associated with dual numbers along with the operations and parents of nodes, allows us to perform the reverse pass necessary in reverse mode AD.  Within the RevDualNumber class we can then create a graph_trace method that uses a topological sorting algorithm to follow each parent DualNumber backwards recursvely as far as possible while still maintaining the correct order for any intermediate dependencies.  This method should return the sorted order of the graph nodes along with the nodes and edges we need to visualize the graph, something we can implement by using Python's graphviz library.  

Eventually, we will be able to utilize the Digraph function from the graphviz package in a separate method called draw_graph which takes the nodes and edges from graph_trace and displays the computational graph.  To actually compute the reverse path, we define a final backwards method that uses the sorted order of graph nodes and the earlier backwards functions.  By calling this method we would compute the reverse pass and return the gradients. Using the reverse mode AD, users could instantiate two dual numbers using RevDualNumber like (x1,x2) = (2,5) so input1 = RevDualNumber(x1), input2 = RevDualNumber(x2).  We could then perform the forward pass by calling our defined function func with res = func(input1,input2) which would return a dual number res. Calling the previously defined method res.backwards() would then output our desired gradient.  The real part of the dual number res gives the value of our function at the point (x1,x2) and res.backwards() gives the gradient of the function func at the point (x1,x2).

#### Additional Considerations
Due to the implementation of reverse mode described above, we will have a modified dual number structure that will enable us to maintain a proper ordering of our nodes in a graph like structure. This is not required for our implementation of forward mode, since we do not need to store the intermediate nodes as this will be more memory-costly. Additionally, our decision to implement a topological sorting algorithm will also improve our efficiency when evaluating our function as compared to a naive implementation.


For cases of $f\colon\mathbb{R}^m\mapsto\mathbb{R} \text{ or } f: \mathbb{R}^m \mapsto\mathbb{R}^n$, our implementation will operate properly. A user should likely elect to use our reverse mode implementation when dealing with a function of high dimensions due to the computational efficiency benefits. Either way, our creation of a function object will take into account dimensional differences between our inputs and outputs, as these are parameters when instantiating such a Func object. The output of our grad() method will return a gradient matrix corresponding to the dimensions of the functions, while the eval() method will determine the corresponding partial derivative.
 

#### Libraries
As stated earlier, we will have to rely on the NumPy library. When we call on sin, cosine, tangent, logarithmic, or exponential functions, it is necessary to include extensions from this library when implementing their definitions. Additionally, the NumPy library will be able to maximize efficiency on the types of our inputs since they may take on the value of a NumPy array. The built-in math library is another viable option, but it results in considerably higher runtime when executed on non-native pythonic types. Our program will also call on the graphviz library to aid us in creating our directed acyclic graphs for reverse mode differentiation. The Digraph method specifically will be employed to visualize the computational order of reverse mode differentiation, which in turn provides a mapped out view of the topological sorting described in this section.


-------

### Licensing

We selected an MIT License for our project due to its simplicity and permissive nature. The straightforward structure of this open-source software license makes it easily readable and understandable for users. Users should also benefit from the permissiveness of the license, allowing free distrubition along with use and modification without restriction for any work released under it. In addition, the MIT license has clear advantages in compatibility. Because it is compatible with copyleft licenses, such as the immensely common GPL, basing our project on the MIT license allows it to be easily integrated with GPL licensed software.  Because it lacks share-alike terms, the MIT license would also enable any derivative works to have differing licenses from our project.  This could make it more popular for use in code that needs to link with free open source software as well as proprietary closed source software.

   One of the major differences between the MIT license and the GNU GPLv3 license is that the former would allow other developers to make and distribute closed source versions of our project while the latter would not.  We feel that our less restrictive MIT license could allow developers to combine code from various applications without violating licensing limitations in a manner that they could not with a GNU GPLv3 license.  This would allow developers to use our project freely and focus solely on creating the best software for consumers – regardless of whether it is open source or proprietary.  Lacking commercial interests, we are also not particularly worried about the lack of explicit patent protection in the MIT license.  Lastly, using an MIT copyright license gives us more freedom in the development process.  If we instead had our code under a GNU GPLv3 license, we would be restricted to only using libraries with copyleft licenses in our project. Surmising these factors, we have opted for the MIT copyright license for the requirements and expected use of our project.


---------

### Feedback

#### Milestone 1

We received feedback on the following three sections:

##### - How to use:
    
  Comments Received:
    
- "I don't think you could initialize a function by using f(x,y) without "def" or "lambda". Please edit that specific line on how users would define their function. This will help you to think more about how you want to structure your functions later on. Additionally, I would suggest using a code cell for the demo to increase the readability. Please use Numpy's documentation as an example of how to create a good demo section. Including the expected output would also be helpful for the users to understand how your program works and what to expect. Please also include an example of how you would expect the users to interact with your reverse mode and generate the computational graph"
    "Please incorporate the suggestions I made about the examples for the next milestone."
    
How we addressed it:
 
    
- We edited the line about how users would define a function to show that they would define their function using "def" or a "lambda" function defintion.  
- Next we added a code cell example that includes input and expected output to provide a demo and increase readability.  In this cell we also included an example of how we would expect users to call the graph attribute of functions in reverse mode to generate a computational graph.


##### - Software Organization:

  Comments Received:
    
- "Great job! Since you mentioned that you would implement a computational graph for the reverse mode, you probably also need a separate py to handle all the graph drawing steps inside your "reverse" folder."
     
How we addressed it:
    
- We added a file called graph.py to our reverse folder which should house the code for the computational graph drawing steps of reverse mode.
- We also decided to add a module operators.py where we plan to instantiate unary operators sin, cos, log, and pow, such that users will be able to generate functions with our implementation of these operators, calling for example ad.sin(). We also include corresponding tests for the above module additions.

##### - License:

  Comments Received:
    
- Conflict with Dependencies?
     
How we addressed it:
    
- We ensured that our dependencies, being numpy and graphviz, would not raise any issues with our MIT license. Numpy uses a liberal BSD license that is compatible with our MIT license, while graphviz utilizes a Common Public License (CPL-1.0), which will also not generate any conflicts with our choice of licensing.